# 1\. Chuẩn bị môi trường

In [1]:
import os
from dotenv import load_dotenv
from pymongo import MongoClient
from sentence_transformers import SentenceTransformer
from pyvi.ViTokenizer import tokenize
import chromadb
from chromadb.config import Settings
from rank_bm25 import BM25Okapi
import numpy as np
from typing import List, Dict, Any, Tuple
import pickle
from pathlib import Path
from tqdm import tqdm

d:\Study_Universtiy\y4semester2\nlp\govbot-nlp-project\pre_retrieval\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [36]:
load_dotenv()

True

In [3]:
mongo_uri = os.getenv('MONGO_URI', 'mongodb://localhost:27017')
mongo_client = MongoClient(mongo_uri)

try:
    mongo_client.admin.command('ismaster')
    print("Kết nối thành công với MongoDB")
except Exception as e:
    print(f"Lỗi kết nối với MongoDB: {e}")

Kết nối thành công với MongoDB


In [4]:
database = mongo_client["govbot"]
procedure_collection = database["bocongan_detailed"]
count = procedure_collection.count_documents({})
print(f"Số lượng documents trong collection: {count}")

# Find a random document
one_document = procedure_collection.find_one({})
print("\nMột document mẫu:")
print(one_document)

Số lượng documents trong collection: 297

Một document mẫu:
{'_id': ObjectId('682c320bed6d3228e1e3a71b'), 'ten': 'Cấp đổi Giấy chứng nhận đủ điều kiện về an ninh, trật tự (thực hiện tại Công an cấp xã)', 'url': 'https://dichvucong.bocongan.gov.vn/bocongan/bothutuc/tthc?matt=52572', 'status': 'pending', 'collected_at': datetime.datetime(2025, 5, 20, 10, 31, 12, 18000), 'ma_thu_tuc': '3.000244', 'linh_vuc': 'Quản lý ngành nghề kinh doanh có điều kiện', 'co_quan_thuc_hien': 'Công an Xã', 'muc_do_cung_cap_dich_vu_cong_truc_tuyen': 'Dịch vụ công trực tuyến một phần', 'cach_thuc_thuc_hien': 'Trực tiếpTrực tuyếnDịch vụ bưu chính', 'trinh_tu_thuc_hien': '- Bước 1: Tổ chức, cá nhân có nhu cầu được cấp đổi Giấy chứng nhận đủ điều kiện về an ninh, trật tự để kinh doanh một số ngành, nghề đầu tư kinh doanh có điều kiện trong những trường hợp bị hư hỏng, sai thông tin, có thay đổi nội dung thông tin ghi trên Giấy chứng nhận đủ điều kiện về an ninh, trật tự hoặc hết thời hạn sử dụng đối với Giấy chứ

# 2\. Embedding Model
Dùng với Transformer để thử nghiệm, đồng thời lấy tokenizer ra để dùng cho BM25

In [5]:
from sentence_transformers import SentenceTransformer

sentences = ["Hà Nội là thủ đô của Việt Nam", "Đà Nẵng là thành phố du lịch"]

model = SentenceTransformer('./vietnamese-embedding')
embeddings = model.encode(sentences)
print("Vector Embeddings:")
print(embeddings)

Vector Embeddings:
[[ 0.19188622  0.56831944 -0.08928554 ...  0.12282588 -0.35222918
   0.41471392]
 [-0.07073209  0.19348213  0.08893326 ...  0.35699594 -0.4206734
  -0.23128642]]


# 3\. BM25

In [ ]:
total_docs = procedure_collection.count_documents({})
print(f"Found {total_docs} documents in MongoDB")
documents = procedure_collection.find({}, {"ten": 1, "linh_vuc": 1, "co_quan_thuc_hien": 1, "_id": 1})

pbar = tqdm(total=total_docs, desc="Processing documents", unit="doc")

documents_text = []
documents_ids = []
documents_names = []

for doc in documents:
    tokenized_text = tokenize(doc["ten"] + " " + doc["linh_vuc"] + " " + doc["co_quan_thuc_hien"])
    documents_text.append(tokenized_text)
    documents_ids.append(str(doc["_id"]))
    documents_names.append(doc["ten"])
    pbar.update(1)




bm25 = BM25Okapi(documents_text)

with open('./bm25/bm25.pkl', 'wb') as f:
    pickle.dump(bm25, f)

with open('./bm25/documents_ids.pkl', 'wb') as f:
    pickle.dump(documents_ids, f)

with open('./bm25/documents_names.pkl', 'wb') as f:
    pickle.dump(documents_names, f)

pbar.close()

Found 297 documents in MongoDB


Processing documents: 100%|██████████| 297/297 [00:00<00:00, 688.95doc/s]


In [10]:
top_k = 15
query = "bằng lái xe"
tokenized_query = tokenize(query.lower())
bm25_scores = bm25.get_scores(tokenized_query)
ranked_indices = np.argsort(bm25_scores)[::-1]
for i in range(top_k):
    print(f"Rank#{i+1} with score {bm25_scores[ranked_indices[i]]}: {documents_names[ranked_indices[i]]}")


Rank#1 with score 8.41148626359777: Thu hồi giấy chứng nhận đăng ký, biển số xe thực hiện bằng dịch vụ công trực tuyến một phần hoặc trực tiếp tại cơ quan đăng ký xe thực hiện tại cấp trung ương, cấp tỉnh, cấp xã
Rank#2 with score 8.161561189734893: Khai báo tạm trú cho người nước ngoài tại Việt Nam bằng Phiếu khai báo tạm trú
Rank#3 with score 7.849384819555349: Xét công nhận liệt sỹ, cấp bằng Tổ quốc ghi công và Giấy chứng nhận gia đình liệt sỹ đối với liệt sỹ và thân nhân liệt sỹ tại Công an cấp tỉnh
Rank#4 with score 7.439766747559665: Huỷ tài khoản điện tử trong trường hợp cơ quan, tổ chức có tài khoản đề nghị bằng văn bản
Rank#5 with score 7.228511586286211: Cấp giấy phép đến các tỉnh, thành phố của Việt Nam cho công dân Lào nhập cảnh bằng Giấy thông hành biên giới tại Công an cấp tỉnh
Rank#6 with score 7.056301035246232: Đăng ký sang tên, di chuyển xe thực hiện tại Công an cấp tỉnh
Rank#7 with score 6.977591915708594: Đăng ký sang tên, di chuyển xe tại Cục
Rank#8 with score 6.95

# 4\. Vector Database với Vietnamese Embedding

In [11]:
chroma_client = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = chroma_client.get_or_create_collection(
    name="procedure_collection",
    metadata={"hnsw:space": "cosine"}
)

In [10]:
documents = procedure_collection.find({}, {"ten": 1, "linh_vuc": 1, "co_quan_thuc_hien": 1, "_id": 1})
embedding_model = SentenceTransformer('./vietnamese-embedding')

pbar = tqdm(total=total_docs, desc="Processing embedding documents", unit="doc")
for index, doc in enumerate(documents):
    text = doc["ten"] + " " + doc["linh_vuc"] + " " + doc["co_quan_thuc_hien"]
    embedding = embedding_model.encode([text])[0]
    chroma_collection.add(
        ids=[str(index)],
        embeddings=[embedding.tolist()],
        documents=[doc["ten"]],
        metadatas=[{"original_id": str(doc["_id"])}]
    )
    pbar.update(1)
pbar.close()

Processing embedding documents: 100%|██████████| 297/297 [00:24<00:00, 12.02doc/s]


In [13]:
embedding_model = SentenceTransformer('./vietnamese-embedding')
query = "bằng lái xe"
embedding = embedding_model.encode([query])[0].tolist()
result = chroma_collection.query(query_embeddings=[embedding], n_results=10)
for i in range(len(result["documents"][0])):
    print(f"Rank#{i+1} with score {1-result['distances'][0][i]}: {result['documents'][0][i]}")

Rank#1 with score 0.5830626487731934: Cấp giấy phép lái xe
Rank#2 with score 0.5236132144927979: Cấp giấy phép cho trung tâm sát hạch lái xe loại 3
Rank#3 with score 0.46963363885879517: Chấp thuận hoạt động của sân tập lái để sát hạch lái xe mô tô
Rank#4 with score 0.4636554718017578: Thu hồi chấp thuận hoạt động của sân tập lái để sát hạch lái xe mô tô
Rank#5 with score 0.44052112102508545: Chấp thuận lại hoạt động của sân tập lái để sát hạch lái xe mô tô
Rank#6 with score 0.41804414987564087: Cấp đổi chứng nhận đăng ký xe, biển số xe thực hiện tại Công an cấp tỉnh
Rank#7 with score 0.4173473119735718: Cấp đổi chứng nhận đăng ký xe, biển số xe thực hiện tại Cục
Rank#8 with score 0.4155663847923279: Cấp lại giấy phép sát hạch cho trung tâm sát hạch lái xe loại 3
Rank#9 with score 0.4106869697570801: Thu hồi chứng nhận đăng ký xe, biển số xe thực hiện tại Công an cấp tỉnh
Rank#10 with score 0.41008466482162476: Cấp lại chứng nhận đăng ký xe, biển số xe thực hiện tại Cục


# 5\. Hybrid Search

In [30]:
import numpy as np
from sklearn.preprocessing import minmax_scale
import pickle

with open('./bm25/bm25.pkl', 'rb') as f:
    bm25 = pickle.load(f)
with open('./bm25/documents_names.pkl', 'rb') as f:
    documents_names = pickle.load(f)
with open('./bm25/documents_ids.pkl', 'rb') as f:
    documents_ids = pickle.load(f)


def hybrid_search(query, bm25, collection, alpha=0.5, top_k=10):
    # BM25
    tokenized_query = tokenize(query.lower())
    bm25_scores = bm25.get_scores(tokenized_query)
    bm25_norm = minmax_scale(bm25_scores).tolist()
    
    # Embedding search từ Chroma
    embedding = embedding_model.encode([query])[0].tolist()
    chroma_result = collection.query(query_embeddings=[embedding], n_results=297)
    
    # Trích xuất điểm embedding từ Chroma
    retrieved_ids = chroma_result['ids'][0]
    emb_scores = chroma_result['distances'][0]
    emb_similarities = [1 - d for d in emb_scores]
    emb_norm = minmax_scale(emb_similarities).tolist()

    idx_to_emb_norm = {retrieved_ids[i]: emb_norm[i] for i in range(len(retrieved_ids))}

    final_scores = []

    for index in range(297):
        bm25_score = bm25_norm[index]
        embeding_score = idx_to_emb_norm[str(index)]
        final_score = alpha * bm25_score + (1 - alpha) * embeding_score
        final_scores.append(final_score)

    ranked_indices = np.argsort(final_scores)[::-1]
    for i in range(top_k):
        print(f"Rank#{i+1} with score {final_scores[ranked_indices[i]]}: {documents_names[ranked_indices[i]]}")

    return {documents_ids[ranked_indices[i]]: documents_names[ranked_indices[i]] for i in range(top_k)}

In [31]:
query = "Giấy phép xuất khẩu pháo ho"
hybrid_search(query, bm25, chroma_collection, alpha=0.3, top_k=10)

Rank#1 with score 0.9633994745076655: Cấp Giấy phép xuất khẩu pháo hoa, thuốc pháo hoa tại Cục Cảnh sát quản lý hành chính về trật tự xã hội - Bộ Công an
Rank#2 with score 0.9377194600666728: Cấp Giấy phép xuất khẩu pháo hoa nổ, thuốc pháo nổ và thiết bị, phụ kiện bắn pháo hoa nổ tại Cục Cảnh sát quản lý hành chính về trật tự xã hội - Bộ Công an
Rank#3 with score 0.9288855966628933: Cấp giấy phép nhập khẩu pháo hoa, thuốc pháo hoa tại Cục Cảnh sát quản lý hành chính về trật tự xã hội - Bộ Công an
Rank#4 with score 0.9061507863122871: Cấp Giấy phép xuất khẩu, nhập khẩu vũ khí tại Cục Cảnh sát quản lý hành chính về trật tự xã hội - Bộ Công an
Rank#5 with score 0.9004475810628655: Cấp Giấy phép nhập khẩu pháo hoa nổ, thuốc pháo nổ và thiết bị, phụ kiện bắn pháo hoa nổ tại Cục Cảnh sát quản lý hành chính về trật tự xã hội - Bộ Công an
Rank#6 with score 0.8800467507577695: Cấp Giấy phép xuất khẩu, nhập khẩu công cụ hỗ trợ thực hiện tại Cục
Rank#7 with score 0.8602381875119547: Cấp Giấy phép

{'682c3222ed6d3228e1e3a77e': 'Cấp Giấy phép xuất khẩu pháo hoa, thuốc pháo hoa tại Cục Cảnh sát quản lý hành chính về trật tự xã hội - Bộ Công an',
 '682c3338e160eafd4442f21f': 'Cấp Giấy phép xuất khẩu pháo hoa nổ, thuốc pháo nổ và thiết bị, phụ kiện bắn pháo hoa nổ tại Cục Cảnh sát quản lý hành chính về trật tự xã hội - Bộ Công an',
 '682c3222ed6d3228e1e3a781': 'Cấp giấy phép nhập khẩu pháo hoa, thuốc pháo hoa tại Cục Cảnh sát quản lý hành chính về trật tự xã hội - Bộ Công an',
 '682c325ded6d3228e1e3a7fb': 'Cấp Giấy phép xuất khẩu, nhập khẩu vũ khí tại Cục Cảnh sát quản lý hành chính về trật tự xã hội - Bộ Công an',
 '682c3233ed6d3228e1e3a7ba': 'Cấp Giấy phép nhập khẩu pháo hoa nổ, thuốc pháo nổ và thiết bị, phụ kiện bắn pháo hoa nổ tại Cục Cảnh sát quản lý hành chính về trật tự xã hội - Bộ Công an',
 '682c321ded6d3228e1e3a765': 'Cấp Giấy phép xuất khẩu, nhập khẩu công cụ hỗ trợ thực hiện tại Cục',
 '682c3233ed6d3228e1e3a7c5': 'Cấp Giấy phép vận chuyển pháo hoa để kinh doanh (thực hiệ

In [29]:
from bson import ObjectId

procedure_collection.find_one({"_id": ObjectId("682c3222ed6d3228e1e3a77e")})

{'_id': ObjectId('682c3222ed6d3228e1e3a77e'),
 'ten': 'Cấp Giấy phép xuất khẩu pháo hoa, thuốc pháo hoa tại Cục Cảnh sát quản lý hành chính về trật tự xã hội - Bộ Công an',
 'url': 'https://dichvucong.bocongan.gov.vn/bocongan/bothutuc/tthc?matt=26455',
 'status': 'pending',
 'collected_at': datetime.datetime(2025, 5, 20, 10, 31, 21, 631000),
 'ma_thu_tuc': '3.000170',
 'linh_vuc': 'Quản lý vũ khí, vật liệu nổ,  công cụ hỗ trợ và pháo',
 'co_quan_thuc_hien': 'Cục cảnh sát quản lý hành chính về trật tự, xã hội',
 'muc_do_cung_cap_dich_vu_cong_truc_tuyen': 'Dịch vụ công trực tuyến toàn trình',
 'cach_thuc_thuc_hien': 'Trực tiếpTrực tuyếnDịch vụ bưu chính',
 'trinh_tu_thuc_hien': 'Bước 1: Cơ quan, tổ chức, doanh nghiệp ở trung ương có nhu cầu cấp giấy phép xuất khẩu pháo hoa, thuốc pháo hoa nộp hồ sơ tại Cục Cảnh sát quản lý hành chính về trật tự xã hội - Bộ Công an vào giờ hành chính các ngày làm việc trong tuần, qua Cổng dịch vụ công trực tuyến hoặc qua dịch vụ bưu chính.Bước 2: Cán bộ t

# 6\. Lọc ứng viên bằng Gemini

In [38]:
from google import genai

gemini_client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

In [39]:
from pydantic import BaseModel

class OutputFormat(BaseModel):
  related_procedures: list[str]

In [40]:
SYSTEM_PROMPT = """
You are a helpful assistant in a Retrieval-Augmented Generation (RAG) system.
Your task is to filter a list of procedures based on the USER QUERY and the dictionary of TOP-10 CANDIDATE PROCEDURES with format {{id: procedure_title}}
You need to analyze the relevance of each procedure title to the user's query and select the most relevant ones.
Return at most 3 IDs of procedures that are clearly relevant to the query. If fewer than 3 are clearly relevant, return fewer. If none are relevant, return an empty list.

# OUTPUT FORMAT: in JSON format:
{{ "related_procedures": ["id1", "id2", "id3"] }}

# EXAMPLE:
- User ask: "Thủ tục xuất khẩu pháo hoa đi nước ngoài"
- Candidate procedures:
{{
  "682c3222ed6d3228e1e3a77e": "Cấp Giấy phép xuất khẩu pháo hoa, thuốc pháo hoa tại Cục Cảnh sát quản lý hành chính về trật tự xã hội - Bộ Công an",
  "682c3338e160eafd4442f21f": "Cấp Giấy phép xuất khẩu pháo hoa nổ, thuốc pháo nổ và thiết bị, phụ kiện bắn pháo hoa nổ tại Cục Cảnh sát quản lý hành chính về trật tự xã hội - Bộ Công an",
  "682c3222ed6d3228e1e3a781": "Cấp giấy phép nhập khẩu pháo hoa, thuốc pháo hoa tại Cục Cảnh sát quản lý hành chính về trật tự xã hội - Bộ Công an",
  "682c325ded6d3228e1e3a7fb": "Cấp Giấy phép xuất khẩu, nhập khẩu vũ khí tại Cục Cảnh sát quản lý hành chính về trật tự xã hội - Bộ Công an",
  "682c3233ed6d3228e1e3a7ba": "Cấp Giấy phép nhập khẩu pháo hoa nổ, thuốc pháo nổ và thiết bị, phụ kiện bắn pháo hoa nổ tại Cục Cảnh sát quản lý hành chính về trật tự xã hội - Bộ Công an",
  "682c321ded6d3228e1e3a765": "Cấp Giấy phép xuất khẩu, nhập khẩu công cụ hỗ trợ thực hiện tại Cục",
  "682c3233ed6d3228e1e3a7c5": "Cấp Giấy phép vận chuyển pháo hoa để kinh doanh (thực hiện tại cấp tỉnh)",
  "682c3222ed6d3228e1e3a779": "Cấp giấy phép vận chuyển pháo hoa để kinh doanh tại Cục Cảnh sát quản lý hành chính về trật tự xã hội - Bộ Công an",
  "682c3215ed6d3228e1e3a746": "Cấp Giấy phép vận chuyển vật liệu nổ công nghiệp thực hiện tại Công an cấp tỉnh",
  "682c3222ed6d3228e1e3a77f": "Cấp giấy phép mua pháo hoa để kinh doanh tại Cục Cảnh sát quản lý hành chính về trật tự xã hội - Bộ Công an"
}}
- Output:
{{
  "related_procedures": [
    "682c3222ed6d3228e1e3a77e",
    "682c3338e160eafd4442f21f"
  ]
}}

BEGIN!
"""

In [41]:
def filter_procedures_with_gemini(user_query, candidates: dict):
    user_prompt = f"""
    # USER QUERY: {user_query}
    # TOP-10 CANDIDATE PROCEDURES: {candidates}
    """
    response = gemini_client.models.generate_content(
        model="gemini-2.0-flash-lite",
        config={
        'response_mime_type': 'application/json',
        'response_schema': OutputFormat,
        "temperature": 0,
        "system_instruction": SYSTEM_PROMPT
        },
        contents=[user_prompt]
    )
    return response.parsed.__dict__

# 7\. Pipeline hoàn chỉnh

In [55]:
from bson import ObjectId

query = "Thời gian giải quyết thủ tục cấp đổi Giấy chứng nhận đủ điều kiện về an ninh, trật tự tại Công an cấp xã và cấp Trung ương có gì khác nhau không?"
candidates = hybrid_search(query, bm25, chroma_collection, alpha=0.3, top_k=10)

related_procedures = filter_procedures_with_gemini(query, candidates)["related_procedures"]

print(related_procedures)

# Query the database for the related procedures
related_procedures_docs = procedure_collection.find({"_id": {"$in": [ObjectId(id) for id in related_procedures]}})
list(related_procedures_docs)




Rank#1 with score 0.9806974337087901: Cấp đổi Giấy chứng nhận đủ điều kiện về an ninh, trật tự (thực hiện tại Công an cấp xã)
Rank#2 with score 0.9675599053328352: Cấp đổi Giấy chứng nhận đủ điều kiện về an ninh, trật tự (thực hiện tại cấp tỉnh)
Rank#3 with score 0.9423406825762455: Cấp lại Giấy chứng nhận đủ điều kiện về an ninh, trật tự (thực hiện tại Công an cấp xã)
Rank#4 with score 0.9318752790005665: Cấp lại Giấy chứng nhận đủ điều kiện về an ninh, trật tự
Rank#5 with score 0.9295614488155634: Cấp đổi Giấy chứng nhận đủ điều kiện về an ninh, trật tự (thực hiện tại cấp Trung ương)
Rank#6 with score 0.9023852047671701: Cấp mới Giấy chứng nhận đủ điều kiện về an ninh, trật tự (thực hiện tại cấp xã)
Rank#7 with score 0.8984195221208964: Cấp lại Giấy chứng nhận đủ điều kiện về an ninh, trật tự (thực hiện tại cấp tỉnh)
Rank#8 with score 0.8963869321461236: Cấp đổi Giấy xác nhận đủ điều kiện kinh doanh dịch vụ phòng cháy và chữa cháy (thực hiện tại cấp tỉnh)
Rank#9 with score 0.89381063

[{'_id': ObjectId('682c320bed6d3228e1e3a71b'),
  'ten': 'Cấp đổi Giấy chứng nhận đủ điều kiện về an ninh, trật tự (thực hiện tại Công an cấp xã)',
  'url': 'https://dichvucong.bocongan.gov.vn/bocongan/bothutuc/tthc?matt=52572',
  'status': 'pending',
  'collected_at': datetime.datetime(2025, 5, 20, 10, 31, 12, 18000),
  'ma_thu_tuc': '3.000244',
  'linh_vuc': 'Quản lý ngành nghề kinh doanh có điều kiện',
  'co_quan_thuc_hien': 'Công an Xã',
  'muc_do_cung_cap_dich_vu_cong_truc_tuyen': 'Dịch vụ công trực tuyến một phần',
  'cach_thuc_thuc_hien': 'Trực tiếpTrực tuyếnDịch vụ bưu chính',
  'trinh_tu_thuc_hien': '- Bước 1: Tổ chức, cá nhân có nhu cầu được cấp đổi Giấy chứng nhận đủ điều kiện về an ninh, trật tự để kinh doanh một số ngành, nghề đầu tư kinh doanh có điều kiện trong những trường hợp bị hư hỏng, sai thông tin, có thay đổi nội dung thông tin ghi trên Giấy chứng nhận đủ điều kiện về an ninh, trật tự hoặc hết thời hạn sử dụng đối với Giấy chứng nhận đủ điều kiện về an ninh, trật t

# 8\. Đánh giá mô hình trích xuất